In [ ]:
# Load packages
library(ggplot2)
library(nlme)
library(mvtnorm)
options(repr.plot.width=3, repr.plot.height=3)

In [ ]:
# Read the VIS snapshot data (with predicted biomass) from file
vis.data = read.table(file="vis_snapshots_with_biomass.csv", sep=",", header=TRUE)

In [ ]:
# Extract S. viridis A10 and S. italica B100, water treatments 100% and 33% full-capacity
vis.subset = vis.data[(vis.data$genotype == 'A10' | vis.data$genotype == 'B100') &
                      (vis.data$treatment == 100 | vis.data$treatment == 33),]

In [ ]:
# Plot fresh-weight biomass for S. viridis and S. italica water treatments 100% and 33% full-capacity
ggplot(vis.subset, aes(x=dap, y=fw_biomass, color=factor(group))) +
    geom_point(size=1) +
    geom_smooth(method="loess",size=1) +
    scale_x_continuous(name="Days after planting") +
    scale_y_continuous(lim=c(-1,41), name="Estimated biomass (g)") +
    theme_bw() +
    theme(legend.position=c(0.25,0.75),
          axis.title.x=element_text(face="bold"),
          axis.title.y=element_text(face="bold")) +
          labs(color="GxM")

In [ ]:
# Growth curve analysis
# First load three functions for non-linear growth curve analysis reproduced from:
# Paine CET, Marthews TR, Vogt DR, Purves D, Rees M, Hector A, Turnbull LA (2012)
# How to fit nonlinear plant growth models and calculate growth rates: an update for ecologists.
# Methods in Ecology and Evolution 3: 245–256. doi: 10.1111/j.2041-210X.2011.00155.x
output.logis.nlsList <- function(fit, times, CI = F, LOG = F, alpha = 0.05){
  coef <- coef(fit)
  params <- transform_param.logis(coef)
  rates <- list()
  groups <- rownames(params)
  n.groups <- nrow(coef)
  
  # compute rates for each group seperately
  rates <- list()
  for(i in 1:(n.groups)){
    K <- params[i,1]; r <- params[i,2]; M0 <- params[i,3]
    rates[[i]] = data.frame(
      times = times,
      M    = (M0*K)/(M0+(K-M0)*exp(-r*times)),
      AGR  = (r*M0*K*(K-M0)*exp(-r*times))/(M0+(K-M0)*exp(-r*times))^2
    )
    rates[[i]]$RGRt <- rates[[i]]$AGR/rates[[i]]$M
    rates[[i]]$RGRm <- r*(1 - rates[[i]]$M/K)
    if(LOG == T){
      rates[[i]]$RGRt <- rates[[i]]$AGR
      rates[[i]]$RGRm <- r*rates[[i]]$M*(1-rates[[i]]$M/K)
      rates[[i]]$AGR  <- rates[[i]]$AGR*exp(rates[[i]]$M)
    }
    # commute CIs for each group's estaimates, if desired
    if(CI == T){
      cov   <- summary(fit)$cov[i,,]
      x <- y <- data.frame(rmvnorm(n=1000, mean=c(coef[i, 1], coef[i, 2], coef[i, 3]), sigma=cov))
      x$K  <- y[,1]
      x$r  <- 1/y[,3]
      x$M0 <- y[,1]/(1 + exp(y[,2]/y[,3])) 
      M <- AGR <- RGRt <- RGRm <- matrix(NA, ncol = length(times), nrow = nrow(x))
      for(j in 1:nrow(x)){
        K <- x[j,4]; r <- x[j,5]; M0 <- x[j,6]
        M[j,]     <- (M0*K)/(M0+(K-M0)*exp(-r*times))
        AGR[j,]   <- (r*M0*K*(K-M0)*exp(-r*times))/(M0+(K-M0)*exp(-r*times))^2
        RGRt[j,]  <- AGR[j,]/M[j,]
        RGRm[j,]  <- r*(1 - M[j,]/K)
        if(LOG ==T){
          RGRt[j,] <- AGR[j,]
          RGRm[j,] <- r*M[j,]*(1 - M[j,]/K)
          AGR[j,]  <- AGR[j,]*exp(M[j,])
        }
      }
    }
    CIs <- summarizer(list(M = M, AGR = AGR, RGRt = RGRt, RGRm = RGRm), alpha)
    rates[[i]]  <-  cbind(rates[[i]], CIs)
  }
  names(rates) <- rownames(params)
  
  # now compute differences among groups
  diffs <- list()
  for(i in 1:(n.groups-1)){
    Ki <- params[i,1]; ri <- params[i,2]; M0i <- params[i,3]
    for(j in (i+1):n.groups){
      Kj <- params[j,1]; rj <- params[j,2]; M0j <- params[j,3]
      diffs.ij = data.frame(
        times = times,
        diffM     = rates[[i]]$M    - rates[[j]]$M,
        diffAGR   = rates[[i]]$AGR  - rates[[j]]$AGR,
        diffRGRt  = rates[[i]]$RGRt - rates[[j]]$RGRt
      )
      # comparing RGRm has to be done on a biomass basis. So it needs special treatment. First, we aev to know what range of biomasses are shared between groups.
      Mmin <- max(min(rates[[i]]$M), min(rates[[j]]$M)) # yieds the range of overlapping masses between groups i & j
      Mmax <- min(max(rates[[i]]$M), max(rates[[j]]$M))
      #diffs.ij$Mseq <- seq(Mmin, Mmax, length = 100)
      diffs.ij$Mseq <- seq(Mmin, Mmax, length = length(times))
      if(LOG == F){
        diffs.ij$diffRGRm  <- ri*(1 - diffs.ij$Mseq/Ki) - rj*(1 - diffs.ij$Mseq/Kj)
      } else{
        diffs.ij$diffRGRm <- ri*diffs.ij$Mseq*(1 - diffs.ij$Mseq/Ki) - rj*Mseq*(1 - diffs.ij$Mseq/Kj)
      }    
      
    }
    if(CI == T){
      # get params for group i
      covi   <- summary(fit)$cov[i,,]
      xi <- yi <- data.frame(rmvnorm(n=1000, mean=c(coef[i, 1], coef[i, 2], coef[i, 3]), sigma=covi))
      xi$K  <- yi[,1]
      xi$r  <- 1/yi[,3]
      xi$M0 <- yi[,1]/(1 + exp(yi[,2]/yi[,3])) 
      
      # get params for group j
      covj   <- summary(fit)$cov[j,,]
      xj <- yj <- data.frame(rmvnorm(n=1000, mean=c(coef[j, 1], coef[j, 2], coef[j, 3]), sigma=covj))
      xj$K  <- yj[,1]
      xj$r  <- 1/yj[,3]
      xj$M0 <- yj[,1]/(1 + exp(yj[,2]/yj[,3])) 
      
      # now compute diffs for each random set of drawn parameters
      Mi <- Mj <- AGRi <- AGRj <- RGRti <- RGRtj <- RGRmi <- RGRmj <- diffM <- diffAGR <- diffRGRt <- diffRGRm <- matrix(NA, ncol = length(times), nrow = nrow(xi))
      for(k in 1:nrow(xi)){
        Ki <- xi[k,4]; ri <- xi[k,5]; M0i <- xi[k,6]
        Kj <- xj[k,4]; rj <- xj[k,5]; M0j <- xj[k,6]
        Mi[k,]    <- (M0i*Ki)/(M0i+(Ki-M0i)*exp(-ri*times))
        Mj[k,]    <- (M0j*Kj)/(M0j+(Kj-M0j)*exp(-rj*times))
        AGRi[k,]  <- (ri*M0i*Ki*(Ki-M0i)*exp(-ri*times))/(M0i+(Ki-M0i)*exp(-ri*times))^2
        AGRj[k,]  <- (rj*M0j*Kj*(Kj-M0j)*exp(-rj*times))/(M0j+(Kj-M0j)*exp(-rj*times))^2
        RGRti[k,] <- AGRi[k,]/Mi[k,]
        RGRtj[k,] <- AGRj[k,]/Mj[k,]
        RGRmi[k,] <- ri*(1 - diffs.ij$Mseq/Ki)
        RGRmj[k,] <- rj*(1 - diffs.ij$Mseq/Kj)
        if(LOG == T){
          RGRti[k,] <- AGRi[k,]
          RGRtj[k,] <- AGRj[k,]
          RGRmi[k,] <- ri*diffs.ij$Mseq*(1 - diffs.ij$Mseq/Ki)
          RGRmj[k,] <- rj*diffs.ij$Mseq*(1 - diffs.ij$Mseq/Kj)
          AGRi[k,]  <- AGRi[k,]*exp(Mi[k,])
          AGRj[k,]  <- AGRj[k,]*exp(Mj[k,])
        }
        diffM[k,]    <- Mi[k,]    - Mj[k,]
        diffAGR[k,]  <- AGRi[k,]  - AGRj[k,]
        diffRGRt[k,] <- RGRti[k,] - RGRtj[k,]
        diffRGRm[k,] <- RGRmi[k,] - RGRmj[k,]
      }
      CIs <- summarizer(list(diffM = diffM, diffAGR = diffAGR, diffRGRt = diffRGRt, diffRGRm = diffRGRm), alpha)
      diffs[[paste(groups[i], groups[j], sep = "_")]]  <-  cbind(diffs.ij, CIs)
    } else{
      diffs[[paste(groups[i], groups[j], sep = "_")]]  <-  diffs.ij
    }
  } # end loop over pairwise combinations of groups
  
  out <- list(params = params, rates = rates, diffs = diffs)
  return(out)
}

transform_param.logis <- function(coef){
  K = coef[1]
  r = 1/(coef[3])
  M0 =  K/(1 + exp(coef[2]/coef[3])) #untransform best-fit parameters to K, r and M0
  if(is.data.frame(K)){
    out <- cbind(K, r, M0)
  } else {
    out <- c(K, r, M0)
  }
  names(out) <- c("K", "r", "M0")
  return(out)
}

# this function returns confidence envelopes around growth trajectories, and growth rates. 
summarizer <- function(dat, alpha){
  n <- length(dat)
  quantiles <- c(alpha/2, 1-(alpha/2))
  CIs <- data.frame(matrix(NA, ncol(dat[[1]]), n*2))
  names(CIs) <- paste(rep(names(dat), each = 2), c("lo", "hi"), sep = ".")
  for(i in 1:n){
    CIs[,(2*i-1):(2*i)] <- t(apply(dat[[i]],    2, quantile, quantiles, na.rm = T))
  }
  return(CIs)
}

In [ ]:
# Group data
vis.grouped = groupedData(fw_biomass ~ dap | group, vis.subset)

In [ ]:
# Fit three-component logistic functions for each genotype x treatment group
fit.logis = nlsList(fw_biomass ~ SSlogis(dap, Asym, xmid, scal), data = vis.grouped)

In [ ]:
# Output estimated growth rate parameters at even time intervals
est.interval = seq(min(vis.subset$dap), max(vis.subset$dap), length = 100)
out.fit.logis = output.logis.nlsList(fit.logis, times = est.interval, CI = TRUE, LOG = FALSE, alpha = 0.05)

In [ ]:
# Plot logistic growth models for *S. viridis* and *S. italica* and absolute growth rates over time
parent.groups = c("A10-100", "A10-33", "B100-100", "B100-33")
group.colors = c("#F8766D", "#7CAE00", "#00BFC4", "#C77CFF")
group.order = names(out.fit.logis$rates)
gca.plot = ggplot(vis.subset, aes(x=dap, y=fw_biomass, color=factor(group))) +
                  geom_point(size=1) +
                  scale_x_continuous(name="Days after planting") +
                  scale_y_continuous(lim=c(-1,41), name="Estimated biomass (g)") +
                  theme_bw() +
                  theme(legend.position=c(0.2,0.8),
                        axis.title.x=element_text(face="bold"),
                        axis.title.y=element_text(face="bold")) +
                  scale_colour_manual(name='GxM',
                                      values=group.colors, labels=parent.groups)

agr.plot = ggplot() + 
           scale_x_continuous(name="Days after planting") +
           scale_y_continuous(name="Absolute growth rate (g/day)") +
           theme_bw() +
           theme(legend.position=c(0.2,0.8),
                 axis.title.x=element_text(face="bold"),
                 axis.title.y=element_text(face="bold")) +
           scale_colour_manual(name='GxM', values=group.colors, labels=parent.groups)

In [ ]:
# Add logistic growth models and confidence intervals
for(g in 1:length(parent.groups)) {
    group = parent.groups[g]
    i = match(group, group.order)
    group.rates = as.data.frame(out.fit.logis$rates[i])
    col.name = gsub('-', '.', group)
    conf.int = data.frame(x=c(group.rates[,paste(col.name, '.times', sep='')],
                          rev(group.rates[,paste(col.name, '.times', sep='')])),
                          y=c(group.rates[,paste(col.name, '.M.lo', sep='')],
                          rev(group.rates[,paste(col.name, '.M.hi', sep='')])))
    agr.conf.int = data.frame(x=c(group.rates[,paste(col.name, '.times', sep='')],
                              rev(group.rates[,paste(col.name, '.times', sep='')])),
                              y=c(group.rates[,paste(col.name, '.AGR.lo', sep='')],
                              rev(group.rates[,paste(col.name, '.AGR.hi', sep='')])))
    gca.plot = gca.plot + geom_polygon(data=conf.int, aes(x=x, y=y), fill='gray60', color=NA, alpha=0.4) +
                          geom_line(data=group.rates, aes_string(x=paste(col.name, '.times', sep=''),
                                           y=paste(col.name, '.M', sep='')), color=group.colors[g])

    agr.plot = agr.plot + geom_polygon(data=agr.conf.int, aes(x=x, y=y), fill='gray60', color=NA, alpha=0.4) +
                          geom_line(data=group.rates, aes_string(x=paste(col.name, '.times', sep=''),
                                           y=paste(col.name, '.AGR', sep='')), color=group.colors[g])
}

In [ ]:
print(gca.plot)

In [ ]:
print(agr.plot)